In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Define wrapper functions

In [5]:
def weight_variable(shape):
    """ Create TensorFlow weight with initial noise. """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """ Create TensorFlow bias with initial value of 0.1. """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    """ 2D TensorFlow convolution with stride of 1 and zero padding. """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """ TensorFlow max pooling over 2x2 blocks. """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                          padding='SAME')

#### Construct network

In [6]:
# Define variables
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
## Layer 1 (Convolutional)

# Create weights
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Reshape image
# -1 allows tf.reshape to infer that dimension
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolve image
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [8]:
## Layer 2 (Convolutional)

# Create weights
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# Convolve
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [9]:
## Layer 3 (Densely Connected)

# Create weights
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# Reshape output from last step into vector
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# Apply weights
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [10]:
## Apply dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [11]:
## Readout layer

# Create weights
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# Compute output
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#### Train network

In [12]:
# Build training function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Build accuracy measure
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:

    # Initialize variables
    sess.run(tf.global_variables_initializer())

    # Iterate through 
    for i in range(20000):
        
        # Import next batch
        # batch = (data, labels)
        # data: 50 x 784, labels: 50 x 10
        batch = mnist.train.next_batch(50)

        # Report iteration stats
        if i % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0],
                                                      y_:batch[1],
                                                      keep_prob: 1})
            print('step {:d}, training accuracy {:g}'.format(
                i, train_accuracy))

        # Train network
        train_step.run(feed_dict={x: batch[0],
                                  y_: batch[1], keep_prob: 0.5})

    # Report final accuracy
    print('test accuracy {:g}'.format(accuracy.eval(
    feed_dict={x: mnist.test.images, y_: mnist.test.labels,
               keep_prob: 1.0})))

step 0, training accuracy 0.1


NameError: name 'noise_batch' is not defined

#### Visualize batch element

In [ ]:
import matplotlib.pyplot as plt
def imshow(img, scale=None):

    # Assign default scale
    if not scale:
        scale = [np.min(img), np.max(img)]

    # Set up image plot
    plt.imshow(img, cmap='gray', vmin=scale[0], vmax=scale[1])
    plt.xticks([]), plt.yticks([])

    # Show
    plt.show()

In [ ]:
imshow(np.reshape(batch[0][1], (28, 28)))